# History Matching

In this tutorial we demonstrate the use of History Matching to determine which points in the input space are plausible given a set of observations. 

Performing History Matching requires:
- a fit emulator that predicts uncertainty (e.g., Gaussian Process) and 
- an observation associated with the simulator output. 

The emulator is used to efficiently estimate the simulator output, accounting for all uncertainties. The emulated output is compared with the observation and parameter inputs that are unlikely to produce the observation can then be “ruled out” as implausible, reducing the input space.

In [1]:
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.simulations.projectile import Projectile
from autoemulate.experimental.calibration.history_matching import HistoryMatching

## 1. Simulate data and train a GP

In [2]:
simulator = Projectile()
x = simulator.sample_inputs(100)
y = simulator.forward_batch(x)

Running simulations: 100%|██████████| 100/100 [00:00<00:00, 773sample/s] 


In [3]:
ae = AutoEmulate(x, y, models=[GaussianProcessExact])

Comparing models: 100%|██████████| 1.00/1.00 [00:04<00:00, 4.22s/model]


In [4]:
model = ae.best_result().model

## 2. Make predictions for new data

For History Matching we need both the predited mean and variance.

In [5]:
x_new = simulator.sample_inputs(10)
pred = model.predict(x_new)
pred_means, pred_vars = pred.mean, pred.variance

## 3. Use History Matching

Instantiate the object with an observed mean and variance for each simulator output.

In [9]:
# Define observed data (mean, variance)
observations = {"distance": (2500, 100)}

hm = HistoryMatching(observations=observations)

Primary use is the `calculate_implausibility` method, which returns the implausibility metric (a number of standard deviations between the emulator mean and the observation) for a number of query points.

In [10]:
implausability = hm.calculate_implausibility(pred_means, pred_vars)
implausability

tensor([[ 3.1149],
        [ 0.4873],
        [16.8701],
        [ 2.3263],
        [ 1.6580],
        [ 3.5040],
        [ 1.8364],
        [ 2.7249],
        [ 1.0752],
        [ 2.2600]])

One can use the implausability scores to filter out the queried parameters and only retain the plausible inputs.

In [8]:
hm.get_nroy(implausability, x_new)

tensor([[-4.8018e+00,  2.5947e+01],
        [ 5.7713e-01,  7.3016e+02],
        [-1.8748e+00,  8.6431e+02],
        [-2.1908e+00,  2.7631e+02],
        [-1.2487e-01,  5.8680e+02],
        [-2.8496e+00,  4.3097e+02],
        [-4.2419e-01,  1.0625e+02]])